In [1]:
import requests
import pandas as pd
from dotenv import load_dotenv
load_dotenv('../05_data_scraping/.env')
import os
# from sqlalchemy import create_engine
# import pymysql
# pymysql.install_as_MySQLdb()
# from dbio import er_to_db, db_connect
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import koreanize_matplotlib
import time
from konlpy.tag import Mecab
mecab = Mecab()
from nltk.probability import FreqDist
from wordcloud import WordCloud
import re
from gensim.models import Word2Vec

/home/user/miniforge3/envs/project/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [19]:
target = 'NNG', 'NNP', 'MAG', 'XR', 'NR', 'VV', 'VA', 'VCP', 'VCN', 'MM', 'IC', 'XPN', 'UNKNOWN'

In [20]:
def string_delete(text) :
    return text.replace("<b>","").replace("</b>", "").replace("&quot;", "")

def parse_date(date_str):
    # 날짜 문자열을 datetime 객체로 변환
    return datetime.strptime(date_str, "%Y%m%d")

def make_df(df_name) :
    for i in df_name :
        for idx in range(0, len(df_name[i])) : 
            df_name[i][idx] = string_delete(df_name[i][idx])
    df_name['postdate'] = df_name['postdate'].apply(parse_date)
    return df_name
    
def extract_words(words_list) : 
    return [word for word, pos in words_list if pos in target]

In [3]:
client_id = os.getenv('client_id') # 환경 변수에 접근
client_pw = os.getenv('client_pw') # 환경 변수에 접근

url = "https://openapi.naver.com/v1/search/blog.json"
final_data_huam = pd.DataFrame()
start = time.time() # 시작
for i in range(1, 1001) : 
    payload = {'query' : '후암동', 'display' : 100, 'start' : i, 'sort' : 'sim'}
    headers = {'X-Naver-Client-Id' : client_id, 'X-Naver-Client-Secret' : client_pw}
    r = requests.get(url, params = payload, headers = headers)
    if r.status_code == 200 :
        data = r.json()
    else :
        print("Error Code:", r.status_code)
        
    
    df_huam_sim = pd.json_normalize(data['items'])
    entire_data_huam_sim = make_df(df_huam_sim)
    final_data_huam = pd.concat([final_data_huam, entire_data_huam_sim], ignore_index=True)
print(final_data_huam)
print(f"{time.time()-start:.4f} sec")

                                       title  \
0          후암동 언뎁트, 밤 티라미수가 너무 맛있는 용산 테라스 카페   
1                   [해방촌/후암동 카페] 콤포트 COMFORT   
2                 후암동 미니네칼국수 반율카페 내돈내산 버스나들이   
3                      용산돼지갈비 정감어린 후암동맛집이네요!   
4           후암동 맛집 호랭이곳간 후암점 서울역 데이트하기 좋은 밥집   
...                                      ...   
99995                     서울역 후암동 이색 카페 홍철책빵   
99996                       후암동 교반탱크파손 보수 현장   
99997  [서울역] 후암동 홍철책빵 / 하고싶은거 다 하는 사람 노홍철 카페   
99998      [후암동 맛집] 풀코스로 먹는 50년 전통의 감자탕, 일미집   
99999  [주말 서울 산책]가을 남산 자락 갈월동과 후암동, 후암시장...    

                                                 link  \
0        https://blog.naver.com/joy_9372/223636837001   
1         https://blog.naver.com/yh02179/223543283113   
2         https://blog.naver.com/comma_c/223515862419   
3          https://blog.naver.com/o_luvp/223557948707   
4         https://blog.naver.com/cchicer/223615505848   
...                                               ...   
99995  https://blog.nave

In [21]:
final_data_huam_pos = final_data_huam.copy()
final_data_huam_pos = final_data_huam_pos[['title', 'postdate']]

In [22]:
final_data_huam_pos['title2'] = final_data_huam_pos['title'].apply(lambda x: mecab.pos(re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", x)))

In [23]:
final_data_huam_pos.head()

,title,postdate,title2
0,"후암동 언뎁트, 밤 티라미수가 너무 맛있는 용산 테라스 카페",2024-10-28,"[(후암동, NNP), (언, VV+ETM), (뎁, NNP), (트, NNG), ..."
1,[해방촌/후암동 카페] 콤포트 COMFORT,2024-08-10,"[(해, NNG), (방촌, NNG), (후암동, NNP), (카페, NNG), (..."
2,후암동 미니네칼국수 반율카페 내돈내산 버스나들이,2024-07-17,"[(후암동, NNP), (미니, NNG), (네, XSN), (칼국수, NNG), ..."
3,용산돼지갈비 정감어린 후암동맛집이네요!,2024-08-23,"[(용산, NNP), (돼지갈비, NNG), (정감, NNG), (어린, VA+ET..."
4,후암동 맛집 호랭이곳간 후암점 서울역 데이트하기 좋은 밥집,2024-10-11,"[(후암동, NNP), (맛집, NNG), (호랭이, NNP), (곳간, NNG),..."


In [25]:
print(type(final_data_huam_pos['title2'][0]))

<class 'list'>


In [28]:
print(type(final_data_huam_pos['title2'][0][0]))

<class 'tuple'>


In [29]:
extract_words(final_data_huam_pos['title2'][0])

['후암동', '뎁', '트', '밤', '티라미수', '너무', '맛있', '용산', '테라스', '카페']

In [30]:
# 특정 형태소만 가져오기
final_data_huam_pos['title2'] = final_data_huam_pos['title2'].apply(lambda x : extract_words(x))
final_data_huam_pos

,title,postdate,title2
0,"후암동 언뎁트, 밤 티라미수가 너무 맛있는 용산 테라스 카페",2024-10-28,"[후암동, 뎁, 트, 밤, 티라미수, 너무, 맛있, 용산, 테라스, 카페]"
1,[해방촌/후암동 카페] 콤포트 COMFORT,2024-08-10,"[해, 방촌, 후암동, 카페, 콤포트]"
2,후암동 미니네칼국수 반율카페 내돈내산 버스나들이,2024-07-17,"[후암동, 미니, 칼국수, 반, 카페, 돈, 내산, 버스, 나들이]"
3,용산돼지갈비 정감어린 후암동맛집이네요!,2024-08-23,"[용산, 돼지갈비, 정감, 후암동, 맛집, 이]"
4,후암동 맛집 호랭이곳간 후암점 서울역 데이트하기 좋은 밥집,2024-10-11,"[후암동, 맛집, 호랭이, 곳간, 후, 암점, 서울역, 데이트, 좋, 밥집]"
...,...,...,...
99995,서울역 후암동 이색 카페 홍철책빵,2024-08-21,"[서울역, 후암동, 이색, 카페, 홍철, 책, 빵]"
99996,후암동 교반탱크파손 보수 현장,2024-10-19,"[후암동, 교반, 탱크, 파손, 보수, 현장]"
99997,[서울역] 후암동 홍철책빵 / 하고싶은거 다 하는 사람 노홍철 카페,2024-10-27,"[서울역, 후암동, 홍, 철책, 빵, 다, 하, 사람, 노홍철, 카페]"
99998,"[후암동 맛집] 풀코스로 먹는 50년 전통의 감자탕, 일미집",2017-04-18,"[후암동, 맛집, 풀코스, 먹, 전통, 감자탕, 미집]"


In [31]:
model_huam_pos = Word2Vec(final_data_huam_pos['title2'], vector_size=100, window=5, min_count=5, workers=8, sg=0)

In [32]:
model_huam_pos_result = model_huam_pos.wv.most_similar("후암", topn = 50)
print(model_huam_pos_result)

[('원두', 0.4170733094215393), ('해', 0.40944015979766846), ('신흥', 0.40699610114097595), ('종로', 0.3912353515625), ('로컬', 0.38407862186431885), ('보이', 0.3815540671348572), ('내공', 0.3756098449230194), ('향미', 0.3725931644439697), ('한적', 0.3609682023525238), ('터줏대감', 0.35582658648490906), ('녹사', 0.35378438234329224), ('종료', 0.35369873046875), ('로맨틱', 0.35258471965789795), ('탑', 0.3496885299682617), ('영업', 0.3492290675640106), ('사진', 0.3481387197971344), ('맨', 0.34452635049819946), ('너무', 0.3418031334877014), ('스터리', 0.33514389395713806), ('특계', 0.33290770649909973), ('동후', 0.32922112941741943), ('틱', 0.32464390993118286), ('술', 0.3246106505393982), ('계획', 0.32115644216537476), ('한눈', 0.320513516664505), ('소개', 0.31815460324287415), ('분식집', 0.3170214593410492), ('소식', 0.3163779675960541), ('오', 0.3160248398780823), ('훨씬', 0.3158344030380249), ('연어', 0.3153960406780243), ('요리', 0.3145737946033478), ('오징어', 0.3139325976371765), ('중화', 0.31386598944664), ('고즈넉', 0.31341519951820374), ('뷰', 0.3132